In [5]:
import os
import json
import requests
import numpy as np
import faiss
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

json_input_data = [
    {
        "camera_id": "Cam-A12",
        "timestamp": "2023-10-01T09:15:23Z",
        "location": "Warehouse Loading Dock",
        "event_class": "Smoke",
        "details": "Light smoke observed near storage crates; possible overheated machine"
    },
    {
        "camera_id": "Cam-B07",
        "timestamp": "2023-10-01T10:03:47Z",
        "location": "Office Lobby Entrance",
        "event_class": "Fire",
        "details": "Flames visible in trash can; small localized fire"
    },
    {
        "camera_id": "Cam-C19",
        "timestamp": "2023-10-02T14:27:59Z",
        "location": "School Parking Lot",
        "event_class": "Assault",
        "details": "Two individuals fighting; bystanders present"
    },
    {
        "camera_id": "Cam-D04",
        "timestamp": "2023-10-02T14:28:20Z",
        "location": "School Parking Lot",
        "event_class": "Assault",
        "details": "Continuation of fight; security intervening"
    },
    {
        "camera_id": "Cam-E22",
        "timestamp": "2023-10-03T07:45:12Z",
        "location": "Hotel Corridor, 2nd Fl",
        "event_class": "Kidnapping",
        "details": "Individual forcibly dragging victim into a side room"
    },
    {
        "camera_id": "Cam-F11",
        "timestamp": "2023-10-03T07:47:31Z",
        "location": "Hotel Lobby Entrance",
        "event_class": "Kidnapping",
        "details": "Possibly related suspect running towards exit"
    },
    {
        "camera_id": "Cam-A12",
        "timestamp": "2023-10-04T16:09:54Z",
        "location": "Warehouse Loading Dock",
        "event_class": "Fire",
        "details": "Sparks and visible flames near machinery"
    },
    {
        "camera_id": "Cam-G56",
        "timestamp": "2023-10-05T01:33:09Z",
        "location": "Parking Garage B3",
        "event_class": "Smoke",
        "details": "Smoke detected near parked vehicle; cause unclear"
    },
    {
        "camera_id": "Cam-H78",
        "timestamp": "2023-10-05T01:35:16Z",
        "location": "Parking Garage B3",
        "event_class": "Fire",
        "details": "Vehicle on fire; smoke now heavier"
    },
    {
        "camera_id": "Cam-D04",
        "timestamp": "2023-10-06T19:02:10Z",
        "location": "City Street, Block 14",
        "event_class": "Assault",
        "details": "Bystander called emergency services; suspect fleeing on foot"
    },
    {
        "camera_id": "Cam-I90",
        "timestamp": "2023-10-07T22:14:37Z",
        "location": "Residential Alley",
        "event_class": "Kidnapping",
        "details": "Victim forced into a black van; partial license plate captured"
    },
    {
        "camera_id": "Cam-J03",
        "timestamp": "2023-10-08T04:58:02Z",
        "location": "Convenience Store",
        "event_class": "Smoke",
        "details": "Smoke near electrical wiring; short circuit suspected"
    },
    {
        "camera_id": "Cam-K22",
        "timestamp": "2023-10-09T13:22:55Z",
        "location": "Apartment Stairwell",
        "event_class": "Assault",
        "details": "Verbal dispute escalating; possible physical interaction"
    },
    {
        "camera_id": "Cam-L01",
        "timestamp": "2023-10-10T11:11:01Z",
        "location": "Mall Food Court",
        "event_class": "Fire",
        "details": "Fire alarm triggered; small blaze in a kitchen area"
    }
]

NVIDIA_API_KEY   = os.environ.get("NVIDIA_API_KEY", "YOUR_NVIDIA_API_KEY")
EMBED_MODEL      = "nvidia/llama-3.2-nv-embedqa-1b-v2"
RERANK_MODEL     = "nvidia/llama-3.2-nv-rerankqa-1b-v2"
LLM_MODEL        = "meta/llama-3.1-70b-instruct"

embedding_client = OpenAI(
    api_key=NVIDIA_API_KEY,
    base_url="https://integrate.api.nvidia.com/v1"
)

llm_client = OpenAI(
    api_key=NVIDIA_API_KEY,
    base_url="https://integrate.api.nvidia.com/v1"
)


def event_to_text(event: dict) -> str:
    return (f"Camera ID: {event['camera_id']}. "
            f"Timestamp: {event['timestamp']}. "
            f"Location: {event['location']}. "
            f"Event Class: {event['event_class']}. "
            f"Details: {event['details']}")

def embed_text(text: str, is_query=False) -> np.ndarray:
    input_type = "query" if is_query else "passage"
    response = embedding_client.embeddings.create(
        input=[text],
        model=EMBED_MODEL,
        encoding_format="float",
        extra_body={"input_type": input_type, "truncate": "NONE"}
    )
    return np.array(response.data[0].embedding, dtype=np.float32)

faiss_index = None
doc_map = {}
next_id = 0

def add_event_to_index(event: dict, event_id: int):
    global faiss_index, doc_map
    

    text_passage = event_to_text(event)
    

    vec = embed_text(text_passage, is_query=False)
    

    if faiss_index is None:
        dim = vec.shape[0]
        index_flat = faiss.IndexFlatL2(dim)
        index_with_ids = faiss.IndexIDMap(index_flat)
        faiss_index = index_with_ids
    

    vec_2d = np.expand_dims(vec, axis=0)
    ids_np = np.array([event_id], dtype=np.int64)
    faiss_index.add_with_ids(vec_2d, ids_np)
    

    doc_map[event_id] = text_passage

for record in json_input_data:
    add_event_to_index(record, next_id)
    next_id += 1

print("FAISS index size:", faiss_index.ntotal)

def retrieve_top_k(query: str, k=3):

    q_vec = embed_text(query, is_query=True)
    q_vec_2d = np.expand_dims(q_vec, axis=0)
    

    distances, ids = faiss_index.search(q_vec_2d, k)
    results = []
    for dist, doc_id in zip(distances[0], ids[0]):
        if doc_id == -1:
            continue
        passage_text = doc_map[doc_id]
    
        results.append((doc_id, passage_text, dist))
    return results

def rerank_passages(query: str, passages: list):
    """
    passages: list of text strings
    Rerank with nvidia/llama-3.2-nv-rerankqa-1b-v2
    """
    url = "https://ai.api.nvidia.com/v1/retrieval/nvidia/llama-3_2-nv-rerankqa-1b-v2/reranking"
    headers = {
        "Authorization": f"Bearer {NVIDIA_API_KEY}",
        "Accept": "application/json"
    }
    payload = {
      "model": RERANK_MODEL,
      "query": {"text": query},
      "passages": [{"text": p} for p in passages]
    }
    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()
    data = response.json()
    

    rankings = data["rankings"]
    

    scored_passages = []
    for r in rankings:
        idx   = r["index"]
        logit = r["logit"]
        text_ = passages[idx]
        scored_passages.append({"text": text_, "score": logit})
    

    scored_passages.sort(key=lambda x: x["score"], reverse=True)
    return scored_passages

def generate_answer(query: str, passages: list, max_tokens=256):
    """
    Use meta/llama-3.1-70b-instruct to produce final text.
    """

    context_text = "\n\n".join(passages)
    system_msg = (
        "You are an AI security event analysis assistant. "
        "You have the following event data:\n\n"
        f"{context_text}\n\n"
        "Answer the user's query based on the above data. "
        "If you're unsure, say 'I don't know'."
    )
    user_msg = query
    
    messages = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": user_msg}
    ]
    
    completion = llm_client.chat.completions.create(
        model=LLM_MODEL,
        messages=messages,
        max_tokens=max_tokens,
        temperature=0.2,
        top_p=0.9
    )
    
    return completion.choices[0].message.content.strip()

user_query = "List any kidnapping events that occurred in a hotel."

raw_results = retrieve_top_k(user_query, k=5)
passages_5 = [r[1] for r in raw_results]

reranked = rerank_passages(user_query, passages_5)
top_passages = [p["text"] for p in reranked[:3]]

final_answer = generate_answer(user_query, top_passages)
print(f"FINAL ANSWER:\n{final_answer}")


FAISS index size: 14
FINAL ANSWER:
 Based on the event data, there are two kidnapping events that occurred in a hotel:

1. Camera ID: Cam-E22, Timestamp: 2023-10-03T07:45:12Z, Location: Hotel Corridor, 2nd Fl.
2. Camera ID: Cam-F11, Timestamp: 2023-10-03T07:47:31Z, Location: Hotel Lobby Entrance.

Both events occurred on the same day, approximately 2 minutes apart, suggesting they may be related.
